In [ ]:
from my_functions import *

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import glob

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_19nov_model11/Fluxes_model_11/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

qso_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
qso_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
qso_zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# Randomly sample sources corresponding to 200 deg2
# idx = np.random.randint(0, 100000, 1)
idx = np.arange(100_000)
qso_flx = qso_flx[:, idx]
qso_err = qso_err[:, idx]
qso_zspec = qso_zspec[idx]

Lya_fts = pd.read_csv('csv/Lya_fts.csv')
EW_qso = np.abs(Lya_fts.LyaEW)[idx] / (qso_zspec + 1)

# Apply errors
np.random.seed(22)
# qso_flx += qso_err * np.random.normal(size=qso_err.shape)

In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-5/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = data.to_numpy()[:, 60 + 1 : 120 + 1].T

# files2 = []
# files3 = []
# for i in range(len(files)):
#     files2.append(f'{filename}SEDs{i + 1}.csv')
#     files2.sort()
#     files3.append(f'{filename}SEDs_no_line{i + 1}.csv')
#     files3.sort()
# fi = []
# for name in files2:
#     fi.append(pd.read_csv(name, header=None))
# fi3 = []
# for name in files3:
#     fi3.append(pd.read_csv(name, header=None))

# mock = {}
# mock['SEDs'] = pd.concat(fi, axis=0, ignore_index=True).to_numpy()
# mock['SEDs_no_line'] = pd.concat(fi3, axis=0, ignore_index=True).to_numpy()
# mock['w_Arr'] = np.load(filename + 'w_Arr.npy')

EW_sf = data['EW0'].to_numpy()
sf_zspec = data['z'].to_numpy()

# sf_L = np.load(f'{filename}L_lya.npy')

In [ ]:
pm_flx = np.hstack((qso_flx, sf_flx))
pm_err = np.hstack((qso_err, sf_err))
zspec = np.concatenate((qso_zspec, sf_zspec))
EW_lya = np.concatenate((EW_qso, EW_sf))

N_sf = sf_flx.shape[1]
N_qso = qso_flx.shape[1]

qso_dL = cosmo.luminosity_distance(qso_zspec).to(u.cm).value
sf_dL = cosmo.luminosity_distance(sf_zspec).to(u.cm).value

sf_L = data['L_lya'].to_numpy()

sf_flambda = 10 ** sf_L / (4*np.pi * sf_dL **2)
qso_flambda = Lya_fts.LyaF[idx] * 1e-17

qso_flambda_relerr = Lya_fts.LyaF_err[idx] * 1e-17 / qso_flambda

qso_L = np.log10(qso_flambda * 4*np.pi * qso_dL ** 2)

L_lya = np.concatenate((qso_L, sf_L))
fline = np.concatenate((qso_flambda, sf_flambda))

is_qso = np.concatenate((np.ones(N_qso), np.zeros(N_sf))).astype(bool)

N_sources = pm_flx.shape[1]

%xdel sf_flx
%xdel sf_err
%xdel qso_flx
%xdel qso_err
%xdel sf_zspec
%xdel qso_zspec
%xdel EW_sf
%xdel EW_qso
%xdel qso_dL
%xdel sf_L
%xdel qso_L
%xdel sf_flambda
%xdel qso_flambda
%xdel mock

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 20)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, pm_err, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    400, obs=True)
other_lines = identify_lines(line_other, pm_flx, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.12

%xdel cont_est_other
%xdel cont_err_other

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

all_mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
all_mag[np.isnan(all_mag)] = 99.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

# i - g
mask = is_qso & (zspec > 2.1) & (zspec < 4)
ax.scatter(mag[mask], (all_mag[-1] - all_mag[-3])[mask], alpha=0.3, color='C0')

mask = ~is_qso & (zspec > 2.1) & (zspec < 4)
ax.scatter(mag[mask], (all_mag[-1] - all_mag[-3])[mask], alpha=0.3, color='C1')

ax.set_xlim((18, 25))
ax.set_ylim((-3, 10))

plt.show()

fig, ax = plt.subplots(figsize=(8, 7))

bins = np.linspace(-4, 10, 30)

mask = is_qso & (zspec > 2.1) & (zspec < 4) & (mag < 23)
ax.hist((all_mag[-1] - all_mag[-3])[mask], color='C0', histtype='step', bins=bins)

mask = ~is_qso & (zspec > 2.1) & (zspec < 4) & (mag < 23)
ax.hist((all_mag[-1] - all_mag[-3])[mask], color='C1', histtype='step', bins=bins)

plt.show()

In [ ]:
mag_min = 16
mag_max = 24

nb_min = 5
nb_max = 15

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
ew_cut = EW_lya > 20
mag_cut = (mag > mag_min) & (mag < mag_max)

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr
)
nice_lya = (nice_lya & z_cut & mag_cut)

In [ ]:
z_Arr = zspec

NB_z_Arr = z_NB(np.arange(56))
lya_lines = np.ones(N_sources).astype(int) * -1
for src in range(N_sources):
    lya_lines[src] = np.argmin(np.abs(z_Arr[src] - NB_z_Arr))

In [ ]:
def EW_L_NB(pm_flx, pm_err, cont_flx, cont_err, z_Arr, lya_lines, F_bias=None,
    nice_lya=None, N_nb=1):
    '''
    Returns the EW0 and the luminosity from a NB selection given by lya_lines
    '''

    w_central = central_wavelength()

    N_sources = pm_flx.shape[1]
    nb_fwhm_Arr = np.array(nb_fwhm(range(56)))

    if nice_lya is None:
        nice_lya = np.ones(N_sources).astype(bool)
    if F_bias is None:
        F_bias = np.ones(60)

    EW_nb_Arr = np.zeros(N_sources)
    EW_nb_e = np.zeros(N_sources)
    L_Arr = np.zeros(N_sources)
    L_e_Arr = np.zeros(N_sources)
    cont = np.zeros(N_sources)
    cont_e = np.zeros(N_sources)
    flambda = np.zeros(N_sources)
    flambda_e = np.zeros(N_sources)

    for src in np.where(nice_lya)[0]: 
        print(f'{src} / {N_sources}')
        l = lya_lines[src]
        if l < 5: continue
        
        cont[src] = cont_flx[l, src]
        cont_e[src] = cont_err[l, src]

        # Let's integrate the NB flux over the transmission curves to obtain Flambda
        l_start = np.max([0, l - N_nb])

        lw = np.arange(l_start, l + N_nb + 1)

        IGM_T_Arr = np.ones(len(lw))
        IGM_T_Arr[: l - l_start] = IGM_TRANSMISSION(w_central[lw[: l - l_start]])
        IGM_T_Arr[l - l_start] = (IGM_TRANSMISSION(w_central[lw[l - l_start]]) + 1) * 0.5

        pm_flx[l_start : l + N_nb + 1, src] /= IGM_T_Arr

        intersec = 0.
        for i in range(lw[0], lw[-1]):
            intersec_dlambda = (
                (nb_fwhm_Arr[i] + nb_fwhm_Arr[i + 1]) * 0.5
                - (w_central[i + 1] - w_central[i])
            )
            intersec += np.min(
                [(pm_flx[i, src]) * intersec_dlambda,
                (pm_flx[i + 1, src]) * intersec_dlambda]
            )

        interval = slice(lw[0], lw[-1] + 1)
        weights = pm_err[interval, src] ** -2
        weights /= weights.max()
        print(weights.shape)
        print()

        flambda_cont = cont[src] * weights * (
            w_central[l + N_nb] + nb_fwhm_Arr[l + N_nb] * 0.5
            - (w_central[l_start] - nb_fwhm_Arr[l_start] * 0.5)
        )

        flambda[src] = np.sum(
            pm_flx[interval, src] * nb_fwhm_Arr[interval] * weights
            ) - intersec - flambda_cont

        flambda_e[src] = (
            np.sum(
                (pm_err[interval, src] * nb_fwhm_Arr[interval] * weights) ** 2
            )
            + (
                cont_e[src] * (
                    w_central[l + N_nb] + nb_fwhm_Arr[l + N_nb] * 0.5
                    - (w_central[l_start] - nb_fwhm_Arr[l_start] * 0.5)
                ) * weights.sum()
            ) ** 2
        ) ** 0.5

    neg_flambda = flambda < 0
    flambda[neg_flambda] = 1e-99
    flambda_e[neg_flambda] = 99.

    flambda /= F_bias[np.array(lya_lines)]

    EW_nb_Arr = flambda / cont / (1 + z_Arr)
    EW_nb_e = flambda_e / cont / (1 + z_Arr)

    LumDist = lambda z: cosmo.luminosity_distance(z).to(u.cm).value
    Redshift = lambda w: w / 1215.67 - 1
    dL = LumDist(z_Arr)
    dL_e = (
        LumDist(
            Redshift(
                w_central[lya_lines] + 0.5 * nb_fwhm_Arr[lya_lines]
            )
        )
        - LumDist(
            Redshift(
                w_central[lya_lines]
            )
        )
    )

    L_Arr = np.log10(flambda * 4*np.pi * dL ** 2)
    L_e_Arr = (
        (4*np.pi * dL ** 2 * flambda_e) ** 2
        + (4*np.pi * dL_e **2 * flambda) ** 2
    ) ** 0.5

    return EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e

def Best_L(pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines):

    EW_nb_Arr1, EW_nb_e1, L_Arr1, L_e_Arr1, flambda1, flambda_e1 = EW_L_NB(
        pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=1
    )

    EW_nb_Arr0, EW_nb_e0, L_Arr0, L_e_Arr0, flambda0, flambda_e0 = EW_L_NB(
        pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=0
    )

    EW_nb_Arr = np.empty(N_sources)
    EW_nb_e = np.empty(N_sources)
    L_Arr = np.empty(N_sources)
    L_e_Arr = np.empty(N_sources)
    flambda = np.empty(N_sources)
    flambda_e = np.empty(N_sources)

    # Where N_nb is best
    best_L1 = (flambda1 / flambda_e1) > (flambda0 / flambda_e0)

    EW_nb_Arr[best_L1] = EW_nb_Arr1[best_L1]
    EW_nb_e[best_L1] = EW_nb_e1[best_L1]
    L_Arr[best_L1] = L_Arr1[best_L1]
    L_e_Arr[best_L1] = L_e_Arr1[best_L1]
    flambda[best_L1] = flambda1[best_L1]
    flambda_e[best_L1] = flambda_e1[best_L1]

    EW_nb_Arr[~best_L1] = EW_nb_Arr0[~best_L1]
    EW_nb_e[~best_L1] = EW_nb_e0[~best_L1]
    L_Arr[~best_L1] = L_Arr0[~best_L1]
    L_e_Arr[~best_L1] = L_e_Arr0[~best_L1]
    flambda[~best_L1] = flambda0[~best_L1]
    flambda_e[~best_L1] = flambda_e0[~best_L1]

    return EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e, best_L1

EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e, best_L1 = Best_L(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines
)

# for nb in nbs_to_consider:
#     nb_bias = np.nanmedian(L_Arr[lya_lines == nb] - L_lya[lya_lines == nb])
#     L_Arr[lya_lines == nb] -= nb_bias

In [ ]:
def plot_contours(maskkk, title='', show=False):
    fig, ax = plt.subplots(figsize=(7, 6))

    mask = maskkk[is_qso] & (qso_flambda_relerr < 0.1)
    Z, x, y = np.histogram2d(
        L_lya[is_qso][mask], L_Arr[is_qso][mask],
        bins=(np.linspace(42, 47, 30), np.linspace(42, 47, 30))
    )

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    N_bins = 10000

    H_Arr = np.linspace(H_min, H_max, N_bins)[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[mask]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr) # 1 sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr) # 2 sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 3 sigma

    ax.contour(
        x_centers, y_centers, Z.T, levels=[H_value_99, H_value_95, H_value_68],
        colors='C0'
    )

    mask = ~is_qso & maskkk
    Z, x, y = np.histogram2d(
        L_lya[mask], L_Arr[mask],
        bins=(np.linspace(42, 47, 30), np.linspace(42, 47, 30))
    )

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    N_bins = 10000

    H_Arr = np.linspace(H_min , H_max , N_bins )[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[ mask ]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr) # 1sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr) # 2sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 2sigma

    ax.contour(
        x_centers, y_centers, Z.T, levels=[H_value_99, H_value_95, H_value_68],
        colors='C1'
    )

    x = np.linspace(40, 48, 100)
    ax.plot(x, x, linestyle='--', color='red', label='1:1')

    ax.set_ylabel('Retrieved $\log L$', fontsize=15)
    ax.set_xlabel('Real $\log L$', fontsize=15)

    ax.set_ylim((42, 47))
    ax.set_xlim((42, 47))

    ax.legend(fontsize=15)

    if len(title) > 0:
        ax.set_title(title, fontsize=20)

    plt.savefig(f'/home/alberto/Desktop/{title}')
    if show:
        plt.show()

In [ ]:
for nb in range(5, 15):
    mask = (mag < 23) & (lya_lines == nb)
    title = f'NB {nb}'
    
    plot_contours(mask, title)
    plot_contours(mask & nice_lya, title + 'nice_lya')
plot_contours((mag < 23) & (lya_lines > 6) & (lya_lines < 20), 'All', True)

In [ ]:
mask = (mag < 23) & (lya_lines == 10)
plt.hist(L_Arr[~is_qso & mask], bins=np.linspace(42, 46, 30), color='C1', alpha=0.4)
plt.hist(L_lya[~is_qso & mask], bins=np.linspace(42, 46, 30), color='C1', histtype='step')

plt.hist(L_Arr[is_qso & mask], bins=np.linspace(42, 46, 30), color='C0', alpha=0.4)
plt.hist(L_lya[is_qso & mask], bins=np.linspace(42, 46, 30), color='C0', histtype='step')
plt.show()

In [ ]:
count_true(best_L1 & ~is_qso)

In [ ]:
print(cont_est_lya[lya_lines, np.arange(N_sources)][~is_qso])
print(flambda[~is_qso])
print(lya_lines[~is_qso])
print(count_true(((flambda < 0))[~is_qso]))

In [ ]:
L_Arr

In [ ]:
flambda